In [27]:
import pandas as pd


timestamp = "run_2020_06_24_10_37_42_ALL_8566794_r3n07"
epoch = "99"
filename = f"/Users/elliotschumacher/Dropbox/git/clel/results/{timestamp}/eval_{epoch}.xlsx"
data_file = "/Users/elliotschumacher/Dropbox/git/clel/datasets/LDC2019T02/tac_kbp_2015_tedl_training_gold_standard_entity_mentions.tab"

results = pd.read_excel(filename)
header_list = ["system_id", "query_id", "mention_string", "doc_id_offsets", "link_id", "entity_type", "mention_type",
               "confidence", "web_search", "wiki_text", "unknown"]
data = pd.read_csv(data_file, sep="\t", names=header_list, header=None,)

data_nils = data[data["link_id"].str.startswith("NIL")]
candidate_list = []
for lang in ["ENG", "SPA", "CMN"]:
    cand_filename = f"/Users/elliotschumacher/Dropbox/git/clel/datasets/LDC2019T02/{lang}/mentions_{lang}_out.csv"
    lang_df = pd.read_csv(cand_filename)
    candidate_list.append(lang_df)
candidate_df = pd.concat(candidate_list)
nil_results = []
for i, row in data_nils.iterrows():
    row_result = results[results["~~mention_uuid"] == row.query_id]
    if len(row_result) > 0:
        lang = row.doc_id_offsets[:3]
        pred_cui = row_result["~pred_cuis_0"]
        correct = pd.isna(pred_cui).iloc[0]
        candidate_titles = candidate_df[candidate_df['id'] == row.query_id]['title'].values.tolist()
        nil_results.append({
            "lang" : lang,
            "doc_id_offsets" : row["doc_id_offsets"],
            "query_id" : row["query_id"],
            "correct" : correct,
            "num_candidates" : len(candidate_titles),
            "_text" : row_result["_text"].iloc[0] ,
            "_sentence" : row_result["_sentence"].iloc[0] ,
            "_candidates" : ",".join([str(x) for x in candidate_titles]),
            "_author" : "author=" in row_result["_sentence"].iloc[0],
            "_author2": "author=\""+row_result["_text"].iloc[0]  in row_result["_sentence"].iloc[0]
        })
        for i in range(10):
            nil_results[-1][f"~pred_cuis_{i}"] = row_result[f"~pred_cuis_{i}"].iloc[0]
nil_df = pd.DataFrame.from_dict(nil_results)
pd.pivot_table(nil_df[(nil_df["correct"] == True) & (nil_df["num_candidates"] > 0)], index=["lang",],
                            values="correct",
                             aggfunc="count").reset_index()

,lang,correct
0,CMN,24
1,ENG,211
2,SPA,20


In [28]:
pd.pivot_table(nil_df[(nil_df["correct"] == True) & (nil_df["num_candidates"] == 0)], index=["lang",],
                            values="correct",
                             aggfunc="count").reset_index()

,lang,correct
0,CMN,372
1,ENG,442
2,SPA,59


In [29]:
pd.pivot_table(nil_df[(nil_df["correct"] == False) & (nil_df["num_candidates"] > 0)], index=["lang",],
                            values="correct",
                             aggfunc="count").reset_index()

,lang,correct
0,CMN,38
1,ENG,265
2,SPA,65


In [30]:
pd.pivot_table(nil_df[(nil_df["correct"] == False) & (nil_df["num_candidates"] == 0)], index=["lang",],
                            values="correct",
                             aggfunc="count").reset_index()

,lang,correct
0,CMN,5
1,ENG,3
2,SPA,2


In [31]:
lang_acc = pd.pivot_table(nil_df[(nil_df["_author"] == True)], index=["lang","correct"],
                            values="_author",
                             aggfunc="count").reset_index()
lang_acc

,lang,correct,_author
0,CMN,True,348
1,ENG,True,634
2,SPA,False,4
3,SPA,True,78


In [32]:
lang_acc = pd.pivot_table(nil_df[(nil_df["_author2"] == True)], index=["lang","correct"],
                            values="_author",
                             aggfunc="count").reset_index()
lang_acc

,lang,correct,_author
0,CMN,True,130
1,ENG,True,605
2,SPA,True,73


In [33]:
options = {}
options['strings_to_formulas'] = False
options['strings_to_urls'] = False
writer = pd.ExcelWriter(f"/Users/elliotschumacher/Dropbox/git/clel/results/{timestamp}/nil.xlsx", engine='xlsxwriter', options=options)

nil_df[(nil_df["correct"] == False)].to_excel(writer, index=False, freeze_panes=(1,0))
writer.save()

In [34]:
for lang in ["ENG", "SPA", "CMN"]:
    corr = len(nil_df[(nil_df["correct"] == True) & (nil_df['lang'] == lang)])
    total = len(nil_df[(nil_df['lang'] == lang)])
    print(f"{lang}, {corr / float(total)}, {corr} / {total}")
    data_lang = data[data["doc_id_offsets"].str.startswith(lang)]
    print(f"{lang}, nil perc { total/ float(len(data_lang) )} - {len(data_lang)}")


ENG, 0.7090119435396308, 653 / 921
ENG, nil perc 0.06801565615537995 - 13541
SPA, 0.541095890410959, 79 / 146
SPA, nil perc 0.034953315776873356 - 4177
CMN, 0.9020501138952164, 396 / 439
CMN, nil perc 0.0334705702958219 - 13116
